In [4]:
import matplotlib.pyplot as plt
import numpy as np 
from math import sqrt 
from math import floor
from math import ceil
from math import exp 
import pickle
from operator import itemgetter
import time
from datetime import timedelta
import matplotlib
from matplotlib import cm, colors
from numpy import amin, amax, ravel
import pandas as pd 
import pickle

In [2]:
def get_all_saws(current_paths, length, current_conformation):
    """ Все созданные конформации лежат в первом аргументе функции get_all_saws, 
    поэтому первым аргументом нужно подавать переменную, которой ранее присвоили пустой список
    
    """
    if(length==1):
        current_paths.append(current_conformation)
    else:
        for step in [(1, 0), (-1, 0), (0, 1),  (0, -1)]:
            new_point = (current_conformation[-1][0]+step[0], current_conformation[-1][1]+step[1] )
            if new_point in current_conformation:
                continue
            else:
                temp_path = current_conformation.copy()
                temp_path.append(new_point)
                get_all_saws(current_paths, length-1,  temp_path)

In [3]:
def end_to_end_distance(path):
    """Евклидова метрика"""
    distance = (path[-1][0]-path[0][0])**2+(path[-1][1]-path[0][1])**2   
    return distance 

In [4]:
steps = [(1, 0), (-1, 0), (0, 1),  (0, -1)]
def xy_energy(saw, seq):
    """Функция принимает на вход конформацию на плоскости(список координат). 
    Возвращает число топологических соседей для нее """
    k=0.0
    for i in range(1, len(saw)-1):
        for step in steps:
            new_point = (saw[i][0]+step[0], saw[i][1]+step[1] )
            if (new_point in saw    ):
                position = saw.index(new_point)
                k=k+np.cos(seq[i]-seq[position])
                
    for step in steps:
        new_point_begin = (saw[0][0]+step[0], saw[0][1]+step[1] )
        new_point_end= ( saw[-1][0]+step[0], saw[-1][1]+step[1])
        
        if(new_point_begin in saw ):
            position = saw.index(new_point_begin)
            k=k+np.cos(seq[0]-seq[position])
 
        if( new_point_end in saw    ):
            position = saw.index(new_point_end)
            k=k+np.cos(seq[-1]-seq[position])
        
        
    #print(k//2)
    return k/2.0

In [ ]:
h = 0
def calc_r (eps, f = 8):

    mean_c_l = []
    
    for i in range(4, f+1):
        
        conformat = []

        start_conformation = [(0,0)]
        get_all_saws(conformat, i ,start_conformation)

        Z = 0.0
        R_sum = 0.0
        
        seq = np.random.uniform(-np.pi,np.pi, (1000, i))
         
        for l in range(len(conformat)):
            r = end_to_end_distance(conformat[l])
            for j in range(len(seq)):
                E = xy_energy(conformat[l], seq[j])
                Z = Z + np.exp(-(-eps*E ))
                 
                R_sum = R_sum + np.exp(-(-eps*E ))*r

        mean_c_l.append(R_sum/Z)
        
 
            
        print(i)
        #print(R_sum,Z)
        print(R_sum/Z)

In [14]:
h = 0
def calc_checks(eps, f = 8, conformat= None):
    if (conformat==None):
        
        conformat = []

        start_conformation = [(0,0)]
        get_all_saws(conformat, i ,start_conformation)

    Z = 0.0
    R_sum = 0.0

    seq = np.random.uniform(-np.pi,np.pi, (1000, f))

    sin = 0.0
    cos = 0.0 
    mag2 = 0.0
    for l in range(len(conformat)):
        r = end_to_end_distance(conformat[l])
        for j in range(len(seq)):
            E = xy_energy(conformat[l], seq[j])
            sin_ = np.mean(np.sin(seq[j]))
            cos_ = np.mean(np.cos(seq[j]))
            mag2_ = np.mean(np.cos(seq[j]))**2+np.mean(np.sin(seq[j]))**2
            Z = Z + np.exp(-(-eps*E ))

            R_sum = R_sum + np.exp(-(-eps*E ))*r
            sin+= (np.exp(-(-eps*E ))*sin_)
            cos+= (np.exp(-(-eps*E ))*cos_)
            mag2+= (np.exp(-(-eps*E ))*mag2_)

 
        #print(R_sum,Z)
    print(R_sum/Z)
    print("sin ", sin/Z)
    print("cos ", cos/Z)
    print("mag2 ", mag2/Z)
 

In [9]:
conformat = []

start_conformation = [(0,0)]
get_all_saws(conformat, 6 ,start_conformation)

In [16]:
calc_checks(0.8, 6, conformat)

9.09945905628017
sin  -0.00995490332543533
cos  -0.017305319039652312
mag2  0.31616627028840927


In [15]:
calc_checks(0.8, 6, conformat)

8.956733635486673
sin  -0.04999729180857924
cos  0.08400457707214998
mag2  0.3762458552720814


In [19]:
data

,,N,J,h,mean_R_sq,err_mean_R_sq,mean_R_gyr_sq,err_mean_R_gyr_sq,mean_e,err_mean_e,mean_e_sq,...,bulk3,err_bulk3,bulk4,err_bulk4,lambda1,err_lambda1,lambda2,err_lambda2,asperical,err_aspherical
NaN,7,1.5,0,9.62586,0.017621,1.67615,0.001582,-0.610896,0.000939,0.421813,0.001188,...,0.000448,0.013398,0.000108,1.3875,0.001742,0.288644,0.000246,0.412975,0.000634,NaN


In [20]:
data = pd.read_csv("CPP/XY_1.500000_0.000000_7_0.txt", sep = " ")
data.iloc[0]

N                    7.000000e+00
J                    1.500000e+00
h                    0.000000e+00
mean_R_sq            9.625860e+00
err_mean_R_sq        1.762100e-02
mean_R_gyr_sq        1.676150e+00
err_mean_R_gyr_sq    1.581920e-03
mean_e              -6.108960e-01
err_mean_e           9.391750e-04
mean_e_sq            4.218130e-01
err_mean_e_sq        1.187760e-03
mean_e_fourth        2.662450e-01
err_mean_e_fourth    1.352930e-03
mean_sin             6.509730e-03
err_mean_sin         4.167390e-03
mean_cos            -1.056820e-02
err_mean_cos         4.341120e-03
mean_m_sq            5.300890e-01
err_mean_m_sq        9.570230e-04
mean_m_fourth        3.441980e-01
err_mean_m_fourth    9.977800e-04
steps                1.000000e+07
bulk2                5.944830e-01
err_bulk2            3.430100e-04
bulk3                1.987650e-01
err_bulk3            4.482580e-04
bulk4                1.339840e-02
err_bulk4            1.082170e-04
lambda1              1.387500e+00
err_lambda1   

In [21]:
data["mean_m_sq"]

0    0.530089
Name: mean_m_sq, dtype: float64